In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, ConcatDataset

import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import json

In [2]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.losses import DiceLoss
from segmentation_models_pytorch.utils.metrics import IoU, Fscore, Accuracy

In [3]:
import matplotlib.pyplot as plt

# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image, 'gray')
    plt.show()

In [4]:
import random

SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

print(np.random.rand(5), torch.randn(5))

[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548 ] tensor([ 1.5410, -0.2934, -2.1788,  0.5684, -1.0845])


# Settings

In [5]:
root = os.getcwd()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

BATCH = 4

GEN_IN_CHANNELS = 1
GEN_N_CLASSES = 1

DIS_IN_CHANNELS = 1
DIS_N_CLASSES = 2

E_IN_CHANNELS = 1
E_N_CLASSES = 2

EPOCH = 200

### Model Settings

#### encoder

In [6]:
ENCODER = "resnet152"
ENCODER_WEIGHT = None

#### decoder

In [7]:
DECODER_ATT = "scse"

#### head

In [8]:
GEN_ACT = "sigmoid"
DIS_ACT = "softmax"
E_ACT = "softmax"

### Optimizer Settings

In [9]:
GEN_OPTIM_NAME = "adam"
GEN_init_lr = 1e-4
GEN_momentum = 0.9

In [10]:
DIS_OPTIM_NAME = "adam"
DIS_init_lr = 1e-3
DIS_momentum = 0.9
DIS_scheduler = "cosineAnnWarm"

In [11]:
E_OPTIM_NAME = "adam"
E_init_lr = 1e-3
E_momentum = 0.9
E_scheduler = "cosineAnnWarm"

# Epochs

In [12]:
from codes.losses import SSIMLoss
from codes.losses import MAELoss
from pytorch_msssim import ssim
from codes.metrics import PSNR, SNR, ContourEval
from codes.activation import Activation
from codes.utils import hu_clip_tensor
from codes.losses import PerceptualLoss
from kornia.filters.sobel import Sobel

In [13]:
def set_requires_grad(nets, requires_grad=False):
    """Set requies_grad=Fasle for all the networks to avoid unnecessary computations
    Parameters:
    nets (network list)   -- a list of networks
    requires_grad (bool)  -- whether the networks require gradients or not
    """
    if not isinstance(nets, list):
        nets = [nets]
        for net in nets:
            if net is not None:
                for param in net.parameters():
                    param.requires_grad = requires_grad

In [14]:
def replace_relu_to_leakyReLU(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.ReLU):
            setattr(model, child_name, nn.LeakyReLU())
        else:
            replace_relu_to_leakyReLU(child)

In [15]:
def replace_bn_to_instanceNorm(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.BatchNorm2d):
            bn = getattr(model, child_name)
            setattr(model, child_name, nn.InstanceNorm2d(bn.num_features))
        else:
            replace_bn_to_instanceNorm(child)

In [16]:
perceptual_ext = PerceptualLoss()

In [17]:
sobel_filter = Sobel().to(device)

In [18]:
@torch.no_grad()
def eval_epoch(generator, tf_discriminator, edge_discriminator, dataloader, device):
  
    # change mode to train and move to current device
    generator = generator.eval().to(device)
    tf_discriminator = tf_discriminator.eval().to(device)
    edge_discriminator = edge_discriminator.eval().to(device)
    
    ssim_ = []
    psnr_ = []
    mae_ = []
    snr_ = []
    
    air_ = []
    bone_ = []
    cont_ = []
 
    tf_acc = []
    edge_acc = []
    
    for index, data in tqdm(enumerate(dataloader)):

        x, y, air_x, bone_x, *_ = data

        x = x.to(device)
        y = y.to(device)
        air_x = air_x.to(device)
        bone_x = bone_x.to(device)

        y_pr = generator(x)
        y_pr = Activation(name=GEN_ACT)(y_pr) # zipped value to [0, 1]
        
        edge_x = sobel_filter(x)
        edge_y_pr = sobel_filter(y_pr)
        edge_y = sobel_filter(y)
        
        total = x.size()[0]
       ############################
        # (1) Eval True/Fake D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################        
        y_cls = tf_discriminator(y)
        for _y_cls in y_cls:
            _y_cls = Activation(name=DIS_ACT)(_y_cls)
            _, _y_cls = torch.max(_y_cls.data, 1)
            correct = (_y_cls == torch.zeros(total, dtype=torch.long, device=device)).sum().item()
            tf_acc += [correct / total]        
        
        y_pr_cls = tf_discriminator(y_pr)
        for _y_pr_cls in y_pr_cls:
            _y_pr_cls = Activation(name=DIS_ACT)(_y_pr_cls)
            _, _y_pr_cls = torch.max(_y_pr_cls.data, 1)
            correct = (_y_pr_cls == torch.ones(total, dtype=torch.long, device=device)).sum().item()
            tf_acc += [correct / total]
        
       ############################
        # (1.1) Eval Edge D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################  
        edge_y_cls = edge_discriminator(edge_y)
        for _edge_y_cls in edge_y_cls:
            _edge_y_cls = Activation(name=DIS_ACT)(_edge_y_cls)
            _, _edge_y_cls = torch.max(_edge_y_cls.data, 1)
            correct = (_edge_y_cls == torch.zeros(total, dtype=torch.long, device=device)).sum().item()
            edge_acc += [correct / total]          
        
        edge_pr_cls = edge_discriminator(edge_y_pr)
        for _edge_pr_cls in edge_pr_cls:
            _edge_pr_cls = Activation(name=DIS_ACT)(_edge_pr_cls)
            _, _edge_pr_cls = torch.max(_edge_pr_cls.data, 1)
            correct = (_edge_pr_cls == torch.ones(total, dtype=torch.long, device=device)).sum().item()
            edge_acc += [correct / total]        
        
        ############################
        # (2) Eval G network: maximize log(D(G(z)))
        ###########################
        
        # main
        _ssim = ssim(y, y_pr, data_range=1.0, size_average=True) # return (N,)
        _psnr = PSNR()(y_pr, y, 1.0)
        _snr = SNR()(y_pr, y)
        _mae = MAELoss()(y_pr.float(), y.float())

        ssim_ += [_ssim.item()]
        psnr_ += [_psnr.item()]
        mae_ += [_mae.item()]
        snr_ += [_snr.item()]
        
        # auxilary
        _min = -500
        _max = 500
        air_window = (-500, -499)
        upper = ((air_window[1]) - (_min))/(_max-(_min))
        lower = ((air_window[0]) - (_min))/(_max-(_min))
        air_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)
        dice = Fscore()(air_pr, air_x)
        air_ += [dice.item()]
        
        bone_window = (255, 256)
        upper = ((bone_window[1]) - (_min))/(_max-(_min))
        lower = ((bone_window[0]) - (_min))/(_max-(_min))
        bone_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)
        dice = Fscore()(bone_pr, bone_x)
        bone_ += [dice.item()]

        for b in range(x.size()[0]):
            cont = ContourEval()(y_pr[b, :, :, :], x[b, :, :, :])
            cont_ += [cont.item()]

    return  sum(ssim_)/len(ssim_), sum(psnr_)/len(psnr_), sum(snr_)/len(snr_), sum(mae_)/len(mae_), \
                    sum(air_)/len(air_), sum(bone_)/len(bone_), sum(cont_)/len(cont_), sum(tf_acc)/len(tf_acc), sum(edge_acc)/len(edge_acc)

In [19]:
@torch.no_grad()
def test_epoch(iid, model, dataloader, device, save=False, path=None):
  
    # change mode to train and move to current device
    model = model.eval().to(device)
    
    for index, data in tqdm(enumerate(dataloader)):

        x, y, air_x, bone_x, *_ = data

        x = x.to(device)
        y = y.to(device)
        air_x = air_x.to(device)
        bone_x = bone_x.to(device)

        y_pr = model(x)
        y_pr = Activation(name=GEN_ACT)(y_pr) # zipped value to [0, 1]
        
        edge_x = sobel_filter(x)
        edge_y_pr = sobel_filter(y_pr)
        edge_y = sobel_filter(y)
        
        _min = -500
        _max = 500
        air_window = (-500, -499)
        upper = ((air_window[1]) - (_min))/(_max-(_min))
        lower = ((air_window[0]) - (_min))/(_max-(_min))
        air_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)
        
        bone_window = (255, 256)
        upper = ((bone_window[1]) - (_min))/(_max-(_min))
        lower = ((bone_window[0]) - (_min))/(_max-(_min))
        bone_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)

        for b in range(x.shape[0]):           
            tmp_y = y[b, :, :, :].unsqueeze(0)
            tmp_y_pr = y_pr[b, :, :, :].unsqueeze(0)
            tmp_air = air_x[b, :, :, :].unsqueeze(0)
            tmp_air_pr = air_pr[b, :, :, :].unsqueeze(0)
            tmp_bone = bone_x[b, :, :, :].unsqueeze(0)
            tmp_bone_pr = bone_pr[b, :, :, :].unsqueeze(0)
            
            _ssim = ssim(tmp_y, tmp_y_pr, data_range=1.0, size_average=True) # return (N,)
            _psnr = PSNR()(tmp_y_pr, tmp_y, 1.0)
            _snr = SNR()(tmp_y_pr, tmp_y)
            _mae = MAELoss()(tmp_y_pr.float(), tmp_y.float())
            _air = Fscore()(tmp_air_pr, tmp_air)
            _bone = Fscore()(tmp_bone_pr, tmp_bone)
            _cont = ContourEval()(tmp_y_pr, x[b, :, :, :])
            
            scores = {
                "ssim score": _ssim.item(),
                "psnr score": _psnr.item(),
                "snr score": _snr.item(),
                "mae error": _mae.item(),
                "air dice score": _air.item(),
                "bone dice score": _bone.item(),
                "contour dice score": _cont.item()
            }
            
            __cbct = (x.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __ct = (tmp_y.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __ct_pred = (tmp_y_pr.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __ct_masked = __ct_pred * tmp_air.squeeze().cpu().numpy()
            __air = (tmp_air.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __air_pr = (tmp_air_pr.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __bone = (tmp_bone.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __bone_pr = (tmp_bone_pr.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __edge_pr = (edge_y_pr[b, :].squeeze().cpu().numpy() * 255).astype(np.uint8)
            __edge_x = (edge_x.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __edge_y = (edge_y.squeeze().cpu().numpy() * 255).astype(np.uint8)
            
#             visualize(
#                 cbct = __cbct,
#                 ct_y = __ct,
#                 ct_pred = __ct_pred,
#                 edge_pred = __edge_pr,
#                 edge_x = __edge_x,
#                 edge_y = __edge_y,
#                 air = __air,
#                 air_pr = __air_pr,
#                 bone = __bone,
#                 bone_pr = __bone_pr
#             )
            
            if save:
                path_dir = os.path.join(path, "file_{}".format(iid))
                try:
                    os.mkdir(path_dir)
                except FileExistsError:
                    pass
                
                cv2.imwrite(os.path.join(path_dir, "cbct.jpg"), __cbct)
                cv2.imwrite(os.path.join(path_dir, "ct.jpg"), __ct)
                cv2.imwrite(os.path.join(path_dir, "ct_pred.jpg"), __ct_pred)
                cv2.imwrite(os.path.join(path_dir, "air.jpg"), __air)
                cv2.imwrite(os.path.join(path_dir, "air_pred.jpg"), __air_pr)
                cv2.imwrite(os.path.join(path_dir, "bone.jpg"), __bone)
                cv2.imwrite(os.path.join(path_dir, "bone_pred.jpg"), __bone_pr)
                cv2.imwrite(os.path.join(path_dir, "edge_pred.jpg"), __edge_pr)
                cv2.imwrite(os.path.join(path_dir, "edge_cbct.jpg"), __edge_x)
                cv2.imwrite(os.path.join(path_dir, "edge_ct.jpg"), __edge_y)
                with open(os.path.join(path_dir, "scores.txt"), "w") as file:
                    file.write(json.dumps(scores))                
            iid += 1

# Discriminator

In [20]:
from codes.activation import Activation
import torchvision.models as models
from codes.losses import MultiScaleHeads

In [21]:
class Discriminator(nn.Module):
    def __init__(self, in_channel=1, n_classes=2, activation=None):
        
        super(Discriminator, self).__init__()
        
        encoder = smp.Unet(encoder_name="resnet18", in_channels=in_channel, classes=n_classes).encoder
        self.encoder = encoder
        self.fc = MultiScaleHeads(n_classes=n_classes, channels=(512, ), activation=None)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.fc([x[-1]])
        return x

# Generator

In [22]:
import segmentation_models_pytorch as smp
from codes.decoder import UnetDecoder
from segmentation_models_pytorch.base.heads import SegmentationHead

In [23]:
class Generator(nn.Module):
    def __init__(self, encoder_name, encoder_weights, in_channels, classes, attention_type):
        
        super(Generator, self).__init__()
        
        self.encoder = smp.Unet(encoder_name=encoder_name, encoder_weights=encoder_weights, in_channels=in_channels, classes=classes).encoder
        self.decoder = UnetDecoder(
                            encoder_channels=[in_channels, 0, 0, 0, 0, 2048],
                            decoder_channels=[512, 256, 128, 64, 16],
                            n_blocks=5,
                            use_batchnorm=True,
                            attention_type=attention_type)
        self.head =  SegmentationHead(
                        in_channels=16,
                        out_channels=classes,
                        activation=None)
        
        
    def forward(self, x):
        x = self.encoder(x)[-1]
        x = self.decoder(*(None, None, None, None, x))
        x = self.head(x)
        
        return x

# Read Data

In [24]:
import glob
from codes.dataset import Dataset, DicomDataset, DicomsDataset
import codes.augmentation as aug

In [149]:
# run_name = wandb.run.name
run_name = "exalted-dream-236"
ELECTRON = False

In [150]:
checkpoint = torch.load(os.path.join("weight-gan", "{}.pth".format(run_name)))

In [151]:
model = smp.Unet(encoder_name=ENCODER, 
                  encoder_weights=ENCODER_WEIGHT, 
                  in_channels=GEN_IN_CHANNELS, 
                  classes=GEN_N_CLASSES,
                 decoder_attention_type=DECODER_ATT)
replace_relu_to_leakyReLU(model)
replace_bn_to_instanceNorm(model)
model.load_state_dict(checkpoint["model"])

<All keys matched successfully>

In [152]:
tf_discriminator = Discriminator(in_channel=DIS_IN_CHANNELS, n_classes=DIS_N_CLASSES, activation=DIS_ACT)
replace_relu_to_leakyReLU(tf_discriminator)
replace_bn_to_instanceNorm(tf_discriminator)
tf_discriminator.load_state_dict(checkpoint["tf_discriminator"])

<All keys matched successfully>

In [153]:
edge_discriminator = Discriminator(in_channel=E_IN_CHANNELS, n_classes=E_N_CLASSES, activation=E_ACT)
replace_relu_to_leakyReLU(edge_discriminator)
replace_bn_to_instanceNorm(edge_discriminator)
edge_discriminator.load_state_dict(checkpoint["edge_discriminator"])

<All keys matched successfully>

# Pelvic

## Pelvic Test CBCT

In [154]:
test_case_path = 'raw/test/*_*'
paths = sorted(glob.glob(test_case_path))

In [155]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [156]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_cbct"))
except FileExistsError:
    pass

In [157]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i],
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="pelvic")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "test_pelvic_cbct", patient_id)) 
    iid += len(scans)

28it [00:05,  5.30it/s]
27it [00:04,  5.46it/s]
26it [00:04,  5.22it/s]
28it [00:04,  5.61it/s]
27it [00:04,  5.46it/s]
28it [00:05,  5.52it/s]
27it [00:04,  5.62it/s]


In [158]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="pelvic")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

191it [00:36,  5.19it/s]


(0.8358015358760095,
 23.324590503233267,
 14.32886245987178,
 0.02825151296108181,
 0.9960922170135182,
 0.9053976769089186,
 0.46576435965393226,
 0.5,
 0.5)

## Pelvic Test CT

In [159]:
test_case_path = 'raw/test/*_*'
paths = sorted(glob.glob(test_case_path))

In [160]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [161]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_ct"))
except FileExistsError:
    pass

In [162]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="pelvic")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "test_pelvic_ct", patient_id)) 
    iid += len(scans)

28it [00:05,  5.26it/s]
27it [00:04,  5.56it/s]
26it [00:04,  5.75it/s]
28it [00:04,  5.79it/s]
27it [00:04,  5.42it/s]
28it [00:05,  5.46it/s]
27it [00:05,  5.39it/s]


In [163]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="pelvic")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

191it [00:35,  5.34it/s]


(0.9369791665002314,
 28.285079286984747,
 19.289351248616324,
 0.015559918766460961,
 0.9965174203321528,
 0.894460730404398,
 0.389444372572824,
 0.5,
 0.5)

## Pelvic L1 CBCT

In [164]:
test_case_path = 'L1_pelvic_processed/reg_pelvic_l1/*_*'
paths = sorted(glob.glob(test_case_path))

In [165]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [166]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_cbct"))
except FileExistsError:
    pass

In [167]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="pelvic")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L1_pelvic_cbct", patient_id)) 
    iid += len(scans)

29it [00:04,  6.14it/s]
26it [00:04,  5.90it/s]
28it [00:04,  6.06it/s]
28it [00:04,  5.93it/s]
27it [00:04,  5.87it/s]
26it [00:04,  5.75it/s]
27it [00:04,  5.80it/s]
30it [00:05,  5.88it/s]
28it [00:04,  5.93it/s]
29it [00:04,  5.98it/s]
29it [00:04,  6.12it/s]
27it [00:04,  6.51it/s]
28it [00:05,  5.55it/s]
26it [00:04,  5.63it/s]
28it [00:04,  5.77it/s]
28it [00:04,  5.63it/s]
27it [00:04,  5.79it/s]
26it [00:04,  5.51it/s]
27it [00:04,  5.85it/s]
29it [00:04,  5.90it/s]
30it [00:05,  5.94it/s]
32it [00:05,  5.79it/s]
28it [00:04,  6.24it/s]
28it [00:04,  5.83it/s]
15it [00:02,  5.34it/s]
28it [00:05,  5.35it/s]
26it [00:04,  5.25it/s]
28it [00:05,  5.40it/s]
26it [00:04,  5.58it/s]
27it [00:05,  5.16it/s]
36it [00:06,  5.83it/s]
28it [00:04,  5.87it/s]
27it [00:04,  5.63it/s]
28it [00:04,  5.80it/s]
30it [00:04,  6.07it/s]
27it [00:04,  5.72it/s]
26it [00:04,  5.88it/s]
29it [00:05,  5.70it/s]
27it [00:04,  5.71it/s]
27it [00:04,  5.73it/s]
28it [00:04,  5.62it/s]
26it [00:04,  5.

In [168]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="pelvic")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

1457it [04:30,  5.39it/s]


(0.8227577859110102,
 22.756960314434526,
 13.555942660255145,
 0.03041113889983912,
 0.9954772157512913,
 0.8762429980774967,
 0.4573121305271651,
 0.5099519560741249,
 0.47151681537405626)

## Pelvic L1 CT

In [169]:
test_case_path = 'L1_pelvic_processed/reg_pelvic_l1/*_*'
paths = sorted(glob.glob(test_case_path))

In [170]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [171]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_ct"))
except FileExistsError:
    pass

In [172]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="pelvic")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L1_pelvic_ct", patient_id)) 
    iid += len(scans)

29it [00:05,  5.26it/s]
26it [00:04,  5.47it/s]
28it [00:05,  5.46it/s]
28it [00:04,  5.74it/s]
27it [00:04,  5.54it/s]
26it [00:04,  5.63it/s]
27it [00:04,  5.91it/s]
30it [00:05,  5.53it/s]
28it [00:04,  5.71it/s]
29it [00:05,  5.16it/s]
29it [00:05,  5.70it/s]
27it [00:04,  5.69it/s]
28it [00:05,  5.35it/s]
26it [00:04,  5.54it/s]
28it [00:05,  5.38it/s]
28it [00:05,  5.42it/s]
27it [00:04,  5.41it/s]
26it [00:04,  5.39it/s]
27it [00:04,  5.95it/s]
29it [00:05,  5.08it/s]
30it [00:05,  5.62it/s]
32it [00:05,  5.46it/s]
28it [00:05,  5.52it/s]
28it [00:04,  5.60it/s]
15it [00:02,  5.41it/s]
28it [00:04,  5.73it/s]
26it [00:05,  5.17it/s]
28it [00:05,  5.30it/s]
26it [00:03,  6.88it/s]
27it [00:05,  5.38it/s]
36it [00:06,  5.26it/s]
28it [00:04,  5.67it/s]
27it [00:04,  5.69it/s]
28it [00:05,  5.55it/s]
30it [00:05,  5.84it/s]
27it [00:04,  5.91it/s]
26it [00:04,  5.68it/s]
29it [00:05,  5.44it/s]
27it [00:04,  5.59it/s]
27it [00:04,  5.51it/s]
28it [00:05,  5.59it/s]
26it [00:04,  5.

In [173]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="pelvic")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

1457it [04:31,  5.36it/s]


(0.9370146152411901,
 28.359386287000625,
 19.15836865458492,
 0.015969035564042888,
 0.9965555423420899,
 0.8791287649891287,
 0.43827183258271657,
 0.49897048730267674,
 0.455044612216884)

## Pelvic L2 CBCT

In [174]:
test_case_path = 'L2_pelvic_processed/reg_pelvic_l2/*_*'
paths = sorted(glob.glob(test_case_path))

In [175]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [176]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_cbct"))
except FileExistsError:
    pass

In [177]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="pelvic")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L2_pelvic_cbct", patient_id)) 
    iid += len(scans)

26it [00:04,  5.48it/s]
26it [00:05,  5.08it/s]
26it [00:04,  5.66it/s]
27it [00:05,  5.32it/s]
31it [00:05,  5.65it/s]
27it [00:04,  5.79it/s]
27it [00:05,  5.32it/s]
26it [00:04,  5.43it/s]
28it [00:05,  5.58it/s]
28it [00:04,  5.62it/s]
27it [00:04,  5.44it/s]
27it [00:04,  5.68it/s]
26it [00:04,  5.51it/s]
26it [00:04,  5.43it/s]
26it [00:04,  5.56it/s]
26it [00:04,  5.64it/s]
26it [00:04,  5.65it/s]
26it [00:04,  5.51it/s]
27it [00:04,  5.63it/s]
26it [00:04,  5.61it/s]
26it [00:04,  5.55it/s]
28it [00:05,  5.55it/s]
28it [00:05,  5.47it/s]
29it [00:05,  5.59it/s]
28it [00:04,  5.75it/s]
26it [00:04,  5.52it/s]
27it [00:04,  5.51it/s]
26it [00:04,  5.47it/s]
28it [00:04,  5.60it/s]
30it [00:05,  5.62it/s]
27it [00:04,  5.69it/s]
29it [00:05,  5.58it/s]
27it [00:04,  5.75it/s]
27it [00:04,  5.60it/s]
26it [00:04,  5.33it/s]
26it [00:04,  5.38it/s]
26it [00:04,  5.86it/s]
26it [00:04,  6.40it/s]
27it [00:04,  5.46it/s]
26it [00:04,  5.51it/s]
26it [00:04,  5.44it/s]
26it [00:04,  5.

In [178]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="pelvic")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

3064it [09:30,  5.37it/s]


(0.8074108277187335,
 22.074450613935685,
 12.489602482836334,
 0.03263976473687095,
 0.9953496138124774,
 0.838104056579603,
 0.4911174575066286,
 0.5205613577023499,
 0.5197454308093995)

## Pelvic L2 CT

In [179]:
test_case_path = 'L2_pelvic_processed/reg_pelvic_l2/*_*'
paths = sorted(glob.glob(test_case_path))

In [180]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [181]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_ct"))
except FileExistsError:
    pass

In [182]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i],
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="pelvic")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L2_pelvic_ct", patient_id)) 
    iid += len(scans)

26it [00:04,  6.23it/s]
26it [00:04,  5.70it/s]
26it [00:04,  5.63it/s]
27it [00:04,  6.08it/s]
31it [00:04,  6.24it/s]
27it [00:04,  5.73it/s]
27it [00:04,  5.97it/s]
26it [00:04,  5.80it/s]
28it [00:04,  6.10it/s]
28it [00:04,  6.42it/s]
27it [00:04,  5.93it/s]
27it [00:04,  6.05it/s]
26it [00:04,  6.07it/s]
26it [00:04,  6.03it/s]
26it [00:04,  5.96it/s]
26it [00:04,  6.14it/s]
26it [00:04,  5.44it/s]
26it [00:04,  5.60it/s]
27it [00:04,  5.70it/s]
26it [00:04,  5.60it/s]
26it [00:04,  5.49it/s]
28it [00:04,  5.62it/s]
28it [00:04,  5.85it/s]
29it [00:05,  5.65it/s]
28it [00:04,  5.68it/s]
26it [00:04,  5.49it/s]
27it [00:04,  5.47it/s]
26it [00:04,  5.37it/s]
28it [00:04,  5.61it/s]
30it [00:05,  5.71it/s]
27it [00:04,  5.51it/s]
29it [00:04,  5.84it/s]
27it [00:04,  5.69it/s]
27it [00:04,  6.39it/s]
26it [00:04,  5.59it/s]
26it [00:05,  5.01it/s]
26it [00:04,  5.65it/s]
26it [00:05,  4.76it/s]
27it [00:05,  5.29it/s]
26it [00:04,  5.20it/s]
26it [00:05,  4.83it/s]
26it [00:05,  4.

In [183]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="pelvic")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

3064it [09:25,  5.42it/s]


(0.9411424490825003,
 29.0512138321879,
 19.46636570060222,
 0.013232706519215813,
 0.9958611560441731,
 0.8823977912206861,
 0.4348477119380516,
 0.4993472584856397,
 0.49510443864229764)

# Abdomen

In [184]:
test_case_path = 'abdomen/test/*_*'
paths = sorted(glob.glob(test_case_path))

## Abdomen on CBCT

In [185]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [186]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_cbct"))
except FileExistsError:
    pass

In [187]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="abdomen")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "abdomen_test_cbct", patient_id)) 
    iid += len(scans)

28it [00:05,  5.30it/s]
78it [00:14,  5.57it/s]
50it [00:08,  5.84it/s]
29it [00:05,  5.51it/s]
50it [00:08,  5.57it/s]
71it [00:12,  5.87it/s]
39it [00:06,  5.63it/s]


In [188]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="abdomen")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

345it [01:03,  5.41it/s]


(0.7272959983867148,
 20.32406278969585,
 10.85470384929491,
 0.044502573697895245,
 0.989591722605515,
 0.7860068054672539,
 0.33695998710134756,
 0.7159420289855073,
 0.5463768115942029)

## Abdomen on CT

In [189]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [190]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_ct"))
except FileExistsError:
    pass

In [191]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="abdomen")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "abdomen_test_ct", patient_id)) 
    iid += len(scans)

28it [00:05,  5.19it/s]
78it [00:13,  5.59it/s]
50it [00:09,  5.49it/s]
29it [00:05,  5.54it/s]
50it [00:09,  5.45it/s]
71it [00:13,  5.46it/s]
39it [00:07,  5.24it/s]


In [192]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="abdomen")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

345it [01:08,  5.07it/s]


(0.9088492580082106,
 27.00215610006581,
 17.53279716104701,
 0.01863579584042663,
 0.9898252428615271,
 0.805966013861356,
 0.3266208920141925,
 0.591304347826087,
 0.3739130434782609)

# Chest

In [193]:
test_case_path = 'chest/test/*_*'
paths = sorted(glob.glob(test_case_path))

## Chest on CBCT

In [194]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [195]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "chest_test_cbct"))
except FileExistsError:
    pass

In [196]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="chest")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "chest_test_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "chest_test_cbct", patient_id)) 
    iid += len(scans)

49it [00:08,  5.86it/s]
39it [00:06,  5.65it/s]
40it [00:06,  5.77it/s]
29it [00:04,  6.29it/s]
35it [00:06,  5.60it/s]
37it [00:07,  5.06it/s]
34it [00:06,  5.47it/s]


In [197]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="chest")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

263it [00:48,  5.41it/s]


(0.7536358060492309,
 20.44078098050542,
 9.99291037606649,
 0.03849360927542806,
 0.9778123118956982,
 0.8222791564446646,
 0.5351129234290395,
 0.6634980988593155,
 0.5570342205323194)

## Chest on CT

In [198]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [199]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "chest_test_ct"))
except FileExistsError:
    pass

In [200]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="chest")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "chest_test_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "chest_test_ct", patient_id)) 
    iid += len(scans)

49it [00:08,  5.86it/s]
39it [00:06,  6.20it/s]
40it [00:06,  5.92it/s]
29it [00:04,  6.08it/s]
35it [00:05,  5.93it/s]
37it [00:06,  5.59it/s]
34it [00:05,  5.89it/s]


In [201]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="chest")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

263it [00:49,  5.31it/s]


(0.9286883242229999,
 28.176404148000273,
 17.728533549000556,
 0.013620279523979576,
 0.978901931413855,
 0.8215224955149306,
 0.4728130947501034,
 0.49619771863117873,
 0.37072243346007605)

# Headneck

In [202]:
test_case_path = 'headneck/test/*_*'
paths = sorted(glob.glob(test_case_path))

## Headneck on CBCT

In [203]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [204]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "hn_test_cbct"))
except FileExistsError:
    pass

In [205]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="headneck")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "hn_test_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "hn_test_cbct", patient_id)) 
    iid += len(scans)

58it [00:09,  6.06it/s]
55it [00:09,  5.99it/s]
55it [00:09,  5.99it/s]
56it [00:08,  6.45it/s]
56it [00:09,  6.02it/s]
55it [00:09,  5.62it/s]
56it [00:09,  5.78it/s]


In [206]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="headneck")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

391it [01:12,  5.39it/s]


(0.83341894064413,
 21.439775281550023,
 7.487222511140282,
 0.028992231377898275,
 0.550826864751007,
 0.746145474305741,
 0.2833169284548677,
 0.43861892583120204,
 0.6560102301790282)

## Headneck on CT

In [207]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [208]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "hn_test_ct"))
except FileExistsError:
    pass

In [209]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="headneck")
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "hn_test_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "hn_test_ct", patient_id)) 
    iid += len(scans)

58it [00:09,  6.03it/s]
55it [00:09,  5.93it/s]
55it [00:09,  5.58it/s]
56it [00:09,  5.93it/s]
56it [00:09,  5.65it/s]
55it [00:09,  5.81it/s]
56it [00:09,  5.81it/s]


In [210]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="headneck")
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

391it [01:13,  5.33it/s]


(0.8913066514465205,
 26.673223134501814,
 12.720670286651767,
 0.022631603734963156,
 0.5999451082038415,
 0.7534508071876542,
 0.2599781919572281,
 0.4731457800511509,
 0.42327365728900257)